Now we parse the content of the JSON file with the QASM circuits

In [1]:
using JSON
using ITensors

# Run only once or gives problems later
include("utils/utils.jl")
using .Utils


In [30]:
json_filename = "subcircuits.json"  
circuits_data, expval_knitting, coefficients = parse_subcircuits(json_filename);

basis_gates = ["h", "rx", "ry", "rz", "rxx","rzz", "ryy", "cx"]
cutoff = 1e-12
maxdim = 1000
method = "densitymatrix" #currently this is not used

all_mpo_sequences, all_rhos, observables = mpo_sequence_apply(circuits_data,basis_gates, cutoff, maxdim, method);

Completed applying MPO sequence for subcircuit B1.
Completed applying MPO sequence for subcircuit B2.
Completed applying MPO sequence for subcircuit B3.
Completed applying MPO sequence for subcircuit B4.
Completed applying MPO sequence for subcircuit B5.
Completed applying MPO sequence for subcircuit B6.
Completed applying MPO sequence for subcircuit B7.
Completed applying MPO sequence for subcircuit B8.
Completed applying MPO sequence for subcircuit B9.
Completed applying MPO sequence for subcircuit B10.
Completed applying MPO sequence for subcircuit B11.
Completed applying MPO sequence for subcircuit B12.
Completed applying MPO sequence for subcircuit B13.
Completed applying MPO sequence for subcircuit B14.
Completed applying MPO sequence for subcircuit B15.
Completed applying MPO sequence for subcircuit B16.
Completed applying MPO sequence for subcircuit B17.
Completed applying MPO sequence for subcircuit B18.
Completed applying MPO sequence for subcircuit B19.
Completed applying MP

In [31]:
expval_tensors = real(sum(coefficients[i] * (sum((inner(observables["A"][j],all_rhos["A"][i]) * inner(observables["B"][j],all_rhos["B"][i])) for j in 1:length(observables["A"]))) for i in 1:length(coefficients)))

discrepancy = abs(round(100*(expval_knitting - expval_tensors)/expval_knitting, digits = 4))

println("""
==============================
• Expected Value Knitting: $(round(expval_knitting, digits = 8))
• Expected Value Tensors:  $(round(expval_tensors, digits = 8))
• Discrepancy    : $discrepancy%
==============================
""")


• Expected Value Knitting: 23.02423374
• Expected Value Tensors:  23.02423349
• Discrepancy    : 0.0%



In [ ]:
# Time of simulation with different bound dimensions

basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cx"]
cutoff = 1e-12
maxdims = 1:2:10

for maxdim in maxdims
    elapsed_time = @elapsed begin
        all_mpo_sequences, all_rhos, observables = redirect_stdout(devnull) do
            mpo_sequence_apply(circuits_data, basis_gates, cutoff, maxdim, method)
        end

        expval_tensors = real(sum(coefficients[i] * sum(inner(observables["A"][j], all_rhos["A"][i]) * inner(observables["B"][j], all_rhos["B"][i]) for j in 1:length(observables["A"])) for i in 1:length(coefficients)))
        discrepancy = abs(round(100 * (expval_knitting - expval_tensors) / expval_knitting, digits = 4))
    end

    println("
    ==============================
    • Expected Value Knitting: $(round(expval_knitting, digits = 8))
    • Expected Value Tensors:  $(round(expval_tensors, digits = 8))
    • Discrepancy    : $discrepancy%
    • Max bound dimension    : $maxdim
    • Elapsed Time : $(round(elapsed_time, digits=2)) seconds
    ==============================
    ")
end



    • Expected Value Knitting: 23.02423374
    • Expected Value Tensors:  21.25002292
    • Discrepancy    : 7.7058%
    • Max bound dimension    : 1
    • Elapsed Time : 70.42 seconds
    

    • Expected Value Knitting: 23.02423374
    • Expected Value Tensors:  23.02423251
    • Discrepancy    : 0.0%
    • Max bound dimension    : 21
    • Elapsed Time : 121.89 seconds
    

    • Expected Value Knitting: 23.02423374
    • Expected Value Tensors:  23.02423349
    • Discrepancy    : 0.0%
    • Max bound dimension    : 41
    • Elapsed Time : 149.3 seconds
    
